In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import torch
import copy
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 랜덤 크롭 및 리사이즈
    transforms.RandomHorizontalFlip(),  # 좌우 뒤집기
    transforms.RandomRotation(15),  # 랜덤 회전
   
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [51]:
# 데이터 로드
train_dataset = datasets.ImageFolder(root='/home/shinds/my document/categorized_data/pad_training_image', transform=transform)    # resized_categorized_data 경로로 수정
val_dataset = datasets.ImageFolder(root='/home/shinds/my document/categorized_data/pad_val_image', transform=transform)    # resized_categorized_data 경로로 수정

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # 원하는 batch_size 사용하시면 됩니다.
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)    # 원하는 batch_size 사용하시면 됩니다.

In [52]:
# 데이터셋의 클래스 수 확인
print(train_dataset.class_to_idx)
print(val_dataset.class_to_idx)

{'athleisure': 0, 'bodyconscious': 1, 'bold': 2, 'cityglam': 3, 'classic': 4, 'disco': 5, 'ecology': 6, 'feminine': 7, 'genderless': 8, 'grunge': 9, 'hiphop': 10, 'hippie': 11, 'ivy': 12, 'kitsch': 13, 'lingerie': 14, 'lounge': 15, 'metrosexual': 16, 'military': 17, 'minimal': 18, 'mods': 19, 'normcore': 20, 'oriental': 21, 'popart': 22, 'powersuit': 23, 'punk': 24, 'space': 25, 'sportivecasual': 26}
{'athleisure': 0, 'bodyconscious': 1, 'bold': 2, 'cityglam': 3, 'classic': 4, 'disco': 5, 'ecology': 6, 'feminine': 7, 'genderless': 8, 'grunge': 9, 'hiphop': 10, 'hippie': 11, 'ivy': 12, 'kitsch': 13, 'lingerie': 14, 'lounge': 15, 'metrosexual': 16, 'military': 17, 'minimal': 18, 'mods': 19, 'normcore': 20, 'oriental': 21, 'popart': 22, 'powersuit': 23, 'punk': 24, 'space': 25, 'sportivecasual': 26}


In [53]:
# ResNet- 18

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=27):
        super(ResNet18, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(BasicBlock, 64, 2)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)


    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# 모델 생성
model = ResNet18()
model = model.to(device)

In [54]:
# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()   # 원하는 손실 함수 사용하시면 됩니다.
optimizer = optim.Adam(model.parameters(), lr=0.001)    # 원하는 옵티마이저와 learning_rate 사용하시면 됩니다.
# 학습률 스케줄러 추가
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

num_epochs = 200  # 원하는 epoch 사용하시면 됩니다.

In [55]:
def train_and_validate_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, patience=5, save_path='/home/shinds/my document/model'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        start_time = time.time()
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)

        # 학습 단계
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_train_samples = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_train_samples += inputs.size(0)

        epoch_loss = running_loss / total_train_samples
        epoch_acc = running_corrects.double() / total_train_samples
        print(f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')

        # 검증 단계
        model.eval()
        val_running_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_inputs.size(0)

                _, val_preds = torch.max(val_outputs, 1)
                val_corrects += torch.sum(val_preds == val_labels.data)
                val_total += val_labels.size(0)

        val_epoch_loss = val_running_loss / val_total
        val_epoch_acc = val_corrects.double() / val_total
        print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Acc: {val_epoch_acc:.4f}')

        # 학습률 스케줄러 단계 수행
        scheduler.step(val_epoch_loss)
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Current Learning Rate: {current_lr:.6f}')

        epoch_duration = time.time() - start_time
        print(f'Epoch {epoch+1} completed in {epoch_duration:.2f} seconds')

        # 최고 성능 모델 저장
        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            no_improve_epochs = 0
            save_file_path = f'{save_path}/best_model.pth'
            torch.save(best_model_wts, save_file_path)
        else:
            no_improve_epochs += 1

        # Early Stopping
        if no_improve_epochs >= patience:
            print(f'Early stopping triggered. No improvement for {patience} epochs.')
            break

    print(f'Best Validation Accuracy: {best_acc:.4f}')
    
    best_model = copy.deepcopy(model)
    best_model.load_state_dict(best_model_wts)
    return best_model, model


In [ ]:
# 학습 실행
best_model, last_model = train_and_validate_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=num_epochs, patience=10, save_path='/home/shinds/my document/model')


Epoch 1/200
--------------------
Train Loss: 3.0936, Train Acc: 0.1143
Validation Loss: 3.0103, Validation Acc: 0.1125
Current Learning Rate: 0.001000
Epoch 1 completed in 6.41 seconds

Epoch 2/200
--------------------
Train Loss: 3.0085, Train Acc: 0.1241
Validation Loss: 3.1524, Validation Acc: 0.0778
Current Learning Rate: 0.001000
Epoch 2 completed in 6.58 seconds

Epoch 3/200
--------------------
Train Loss: 2.9838, Train Acc: 0.1241
Validation Loss: 3.0481, Validation Acc: 0.1073
Current Learning Rate: 0.001000
Epoch 3 completed in 6.50 seconds

Epoch 4/200
--------------------
Train Loss: 2.9741, Train Acc: 0.1219
Validation Loss: 3.0090, Validation Acc: 0.0841
Current Learning Rate: 0.001000
Epoch 4 completed in 6.42 seconds

Epoch 5/200
--------------------
Train Loss: 2.9496, Train Acc: 0.1319
Validation Loss: 2.9712, Validation Acc: 0.1115
Current Learning Rate: 0.001000
Epoch 5 completed in 6.85 seconds

Epoch 6/200
--------------------
Train Loss: 2.9546, Train Acc: 0.127